In [ ]:
#Packages
%pylab
%matplotlib inline
import numpy as np
import scipy.ndimage
import matplotlib.pyplot as plt
from images_segmentation import otsu as ot

In [ ]:
# functions that still have to be defined: otsu_t(); otsu_i() and crop(image, first pixel coordinates, last pixel coordinates)

In [ ]:
#sliding window average treshold method
def t_sw(image, stepsize, framesize):
    img = copy(image)
    array = np.empty([img.shape[0],img.shape[1]], dtype = list)
    x=0
    y=0
    while y+framesize<=img.shape[1]:
        while x+framesize<=img.shape[0]:
            window=crop(img, (x,y), (x+framesize, y+framesize))
            otsu_t(window)
            for a in window.shape[0]:
                for b in window.shape[1]:
                    a+=x
                    b+=y
                    array[a,b].append(treshold)
                x+=stepsize
            y+=stepsize
            for i in img.shape[0]:
                for j in img.shape[1]:
                    t=sum(array[x,y])/len(array[x,y])
                    if img[x,y]<=t:
                        img[x,y]=0
                    else:
                        img[x,y]=255
                    return img



In [ ]:
#sliding window foreground background assignment
def i_sw(image,stepsize,framesize):
    img=copy(image)
    array=np.empty([img.shape[0],img.shape[1]], dtype=list)
    x=0
    y=0
    while y+framesize<=img.shape[1]:
        while x+framesize<=img.shape[0]:
            window=crop(img, (x,y), (x+framesize, y+framesize))
            otsu_i(window)
            for a in window.shape[0]:
                for b in window.shape[1]:
                    c=a+x
                    d=b+y
                    array[c,d].append(intensity[a,b])
            x+=stepsize
        y+=stepsize
    for i in img.shape[0]:
        for j in img.shape[1]:
            a=mode(array[x,y])
            if a=='f':
                img[x,y]=255
            else:
                img[x,y]=0
    return img
                
